In [1]:
#importing the necessary modules

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import torchvision
import torchvision.transforms as transforms

In [2]:
#creating our hybrid model which can accept both image and tabular data

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        #for Image data 

  #first let us identify the mnist number from mnist image

        # input 28 # output 24 # receptive_field = 5
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5)
        #relu=> maxpool =>24*24*20=> output =>12*12*20 RF=10*10

        # input 12 # output 8 # receptive_field = 14*14
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=5)
        #relu=> maxpool =>8*8*40=> output =>4*4*40 RF=28*28


        #input 4*4*50 output = 100
        self.fc1 = nn.Linear(4*4*40, 100)#we have taken batch size of 100



        ## for tabular data, to sum our random number
        
        self.tabular_fc_rn= nn.Linear(110, 200) 
        self.fc3 = nn.Linear(200, 50) 
        #converting it to one hot encoded output
        self.fc4 = nn.Linear(50,29)

    def forward(self, img,tab):
        img = F.relu(self.conv1(img))
        img = F.max_pool2d(img, 2, 2)
        img = F.relu(self.conv2(img))
        img = F.max_pool2d(img, 2, 2)
        #we are reshaping our tensor in order to flatten our layer into fully connected layer
        # We are reshaping it to have 4*4*40 number of columns and 
        #telling Pytorch to decide the number of rows by itself by giving -1 as parameter 
        img = img.view(-1, 4*4*40)
        img = self.fc1(img)

        # combining mnist image with tabular data random number
        x = torch.cat((img, tab), dim=1)

        #oassing it to the network to recognize and sum the data
        x = self.fc4(F.relu(self.fc3(F.relu(self.tabular_fc_rn(x)))))
        img = x[:,0:10]
        tab = x[:,10:]
        

        #returning  mnist number and sum of random numbr
        return  F.log_softmax(img, dim=1),F.log_softmax(tab, dim=1)


In [3]:

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [4]:
device

device(type='cuda')

In [8]:
from torch.utils.data import Dataset
import torch.nn.functional as F

#creating  a custom dataset for training a network with 2 inputs

class MyDataset_train(Dataset):
    def __init__(self, tabular_data):
        #Initializing data
        self.RandomNumberdata = tabular_data
        self.MNISTImagedata = torchvision.datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))
        
    def __getitem__(self, index):
        r = self.RandomNumberdata[index]
        RandomNumber = F.one_hot(r, num_classes = 10).type(torch.float32).requires_grad_(True)
        
        MNISTimage, MNISTlabel = self.MNISTImagedata[index] 
        
        # for random number label is the number itelf
        # we are creating a label for our network to get trained to compute sum
        Sum_label = r.item() + MNISTlabel 

        #returning multiple items
        return RandomNumber, Sum_label, MNISTimage, MNISTlabel
    
    def __len__(self):
        return len(self.RandomNumberdata)

In [9]:
# creating a tensor of random numbers with a size equal to that of train data
random_number_tensor = torch.randint(low=0,high=9,size=(60000,))

myRNMNIST_train = MyDataset_train(random_number_tensor)

In [10]:
batch_size = 100
#declared a batch size of 100
train_RNMNIST_loader = torch.utils.data.DataLoader(myRNMNIST_train, batch_size = batch_size, shuffle=True)

In [12]:
# creating a dataset for training
class MyDataset_test(Dataset):
    def __init__(self, data_array):
        self.RandomNumberdata = data_array
        self.MNISTdata = torchvision.datasets.MNIST('../data', train=False, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))
        
    def __getitem__(self, index):
        r = self.RandomNumberdata[index]
        #performing one hot encoding to faster theadding of two numbers training
        RandomNumber = F.one_hot(r, num_classes = 10).type(torch.float32)

        MNISTimage, MNISTlabel = self.MNISTdata[index] 
        
       #creating sum label
        SUMlabel = r.item() + MNISTlabel 
        
        return RandomNumber, SUMlabel, MNISTimage, MNISTlabel
    
    def __len__(self):
        return len(self.RandomNumberdata)

In [13]:
#generating random number tensor
random_tensor_test = torch.randint(low=0,high=9,size=(10000,))

myRNMNIST_test = MyDataset_test(random_tensor_test)

test_RNMNIST_loader = torch.utils.data.DataLoader(myRNMNIST_test, batch_size = batch_size, shuffle=True)

In [15]:
from tqdm import tqdm
#creating functions for traing and testing our model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (rn_number, rn_sum_label, mnist_image, mnist_label) in enumerate(pbar):

        # TRAINING ON GPU
        rn_number, rn_sum_label = rn_number.to(device), rn_sum_label.to(device)
        mnist_image, mnist_label = mnist_image.to(device), mnist_label.to(device)
        
        optimizer.zero_grad()

        output_image, output_rnd_sum = model(mnist_image, rn_number)
        
        loss_mnist_img = F.nll_loss(output_image, mnist_label) # LOSS FOR PREDICTING MNIST LABEL
        loss_sum_rn_mn = F.nll_loss(output_rnd_sum, rn_sum_label)# LOSS FOR PREDICTING SUM LABEL
        
        #adding both losses as overall loss
        loss = loss_mnist_img + loss_sum_rn_mn
        loss.backward()
        optimizer.step()
        #using progress bar to show output
        pbar.set_description(desc= f'MNIST loss={loss_mnist_img.item()} SUM loss={loss_sum_rn_mn.item()} batch_id={batch_idx}')

def test(model, device, test_loader):
    model.eval()
    test_loss_mnist = 0
    test_loss_sum = 0
    correct_mnist = 0
    correct_sum = 0
    with torch.no_grad():
        for rn_number, rn_sum_label, mnist_image, mnist_label in test_loader:
            
            rn_number, rn_sum_label = rn_number.to(device), rn_sum_label.to(device)
            mnist_image, mnist_label = mnist_image.to(device), mnist_label.to(device)
            
            output_image, output_rnd_sum = model(mnist_image, rn_number)
            
            test_loss_mnist += F.nll_loss(output_image, mnist_label, reduction='sum').item()
            test_loss_sum += F.nll_loss(output_rnd_sum, rn_sum_label, reduction='sum').item()

            pred_mnist = output_image.argmax(dim=1, keepdim=True)  
            pred_sum = output_rnd_sum.argmax(dim=1, keepdim=True)
            
            correct_mnist += pred_mnist.eq(mnist_label.view_as(pred_mnist)).sum().item()
            correct_sum += pred_sum.eq(rn_sum_label.view_as(pred_sum)).sum().item()

    test_loss_mnist /= len(test_loader.dataset)
    test_loss_sum /= len(test_loader.dataset)
  #printing losses and efficiency
  
    print('\nTest set: Average MNIST loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss_mnist, correct_mnist, len(test_loader.dataset),
        100. * correct_mnist / len(test_loader.dataset)))
    
    print('\nTest set: Average SUM loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss_sum, correct_sum, len(test_loader.dataset),
        100. * correct_sum / len(test_loader.dataset)))

In [16]:
network = Network().to(device)
optimizer = optim.SGD(network.parameters(), lr=0.01, momentum=0.9)

torch.set_printoptions(edgeitems=11)


for epoch in range(1, 11):
    train(network, device, train_RNMNIST_loader, optimizer, epoch)
    test(network, device, test_RNMNIST_loader)

  0%|          | 0/600 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
MNIST loss=0.11605943739414215 SUM loss=2.2462875843048096 batch_id=599: 100%|██████████| 600/600 [00:26<00:00, 23.05it/s]



Test set: Average MNIST loss: 0.0722, Accuracy: 9789/10000 (98%)


Test set: Average SUM loss: 2.2212, Accuracy: 1526/10000 (15%)



MNIST loss=0.07319030910730362 SUM loss=1.151845932006836 batch_id=599: 100%|██████████| 600/600 [00:25<00:00, 23.46it/s]



Test set: Average MNIST loss: 0.0494, Accuracy: 9848/10000 (98%)


Test set: Average SUM loss: 1.1579, Accuracy: 5198/10000 (52%)



MNIST loss=0.02641809731721878 SUM loss=0.6257476806640625 batch_id=599: 100%|██████████| 600/600 [00:25<00:00, 23.46it/s]


...


Test set: Average MNIST loss: 0.0289, Accuracy: 9903/10000 (99%)


Test set: Average SUM loss: 0.0609, Accuracy: 9870/10000 (99%)



MNIST loss=0.002785096876323223 SUM loss=0.019118690863251686 batch_id=599: 100%|██████████| 600/600 [00:25<00:00, 23.85it/s]



Test set: Average MNIST loss: 0.0315, Accuracy: 9911/10000 (99%)


Test set: Average SUM loss: 0.0589, Accuracy: 9867/10000 (99%)



MNIST loss=0.003368350211530924 SUM loss=0.025412114337086678 batch_id=599: 100%|██████████| 600/600 [00:25<00:00, 23.82it/s]



Test set: Average MNIST loss: 0.0288, Accuracy: 9903/10000 (99%)


Test set: Average SUM loss: 0.0471, Accuracy: 9890/10000 (99%)

